# 1. Gerekli Kütüphanelerin Yüklenmesi

Bu bölümde, derin öğrenme modelimizi geliştirmek ve veri setini işlemek için gerekli olan temel kütüphaneleri yüklüyoruz.
* **torch & torchvision:** PyTorch derin öğrenme çerçevesi ve görüntü işleme araçları.
* **transforms:** Görüntüleri tensörlere çevirmek ve veri çoğaltma (augmentation) işlemleri için.
* **pandas:** Test veri setindeki etiketlerin bulunduğu CSV dosyasını okumak için.
* **PIL & pathlib:** Görüntü dosyalarını açmak ve dosya yollarını yönetmek için.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
from PIL import Image
from pathlib import Path

# 2. Özel Test Veri Seti Sınıfı (Custom Dataset)

GTSRB veri setinin **Test** kısmı, Eğitim kısmından farklı bir yapıya sahiptir. Eğitim verileri sınıflara göre klasörlenmişken, Test verileri tek bir klasörde karışık haldedir ve etiketleri bir CSV dosyasında tutulur.

Bu nedenle, PyTorch'un standart `ImageFolder` yapısını test verisinde kullanamayız. Aşağıdaki `GTSRBTestDataset` sınıfı:
1.  CSV dosyasından dosya isimlerini ve sınıf ID'lerini okur.
2.  `__getitem__` fonksiyonu ile her bir görüntüyü diskten yükler ve ilgili dönüşümleri (transform) uygular.

In [2]:
# --- 3. Test Verisi için Özel Dataset Sınıfı ---
class GTSRBTestDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        try:
            self.annotations = pd.read_csv(csv_file, delimiter=';')
        except Exception as e:
            print(f"HATA: '{csv_file}' okunurken sorun: {e}")
            raise e
            
        self.img_dir = img_dir
        self.transform = transform
        
        try:
            self.image_filenames = self.annotations['Filename']
            self.labels = self.annotations['ClassId']
        except KeyError:
            print(f"HATA: CSV dosyasında 'Filename' veya 'ClassId' sütunları bulunamadı.")
            print("Lütfen CSV dosyanızı kontrol edin.")
            raise

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = self.image_filenames[index]
        img_path = self.img_dir / img_name
        
        try:
            image = Image.open(img_path).convert('RGB')
        except FileNotFoundError:
            print(f"HATA: '{img_path}' resmi bulunamadı.")
            return None, None # Hata durumunda None döndür (DataLoader'da işlenir)

        label = int(self.labels[index])
        
        if self.transform:
            image = self.transform(image)
            
        return (image, label)

# 3. Veri Hazırlama ve Yükleme Fonksiyonu (`get_data_loaders`)

Bu fonksiyon, veri işleme boru hattının (pipeline) tamamını yönetir ve modele beslenecek `DataLoader` nesnelerini üretir.

**Fonksiyonun gerçekleştirdiği temel adımlar:**
1.  **Dosya Yolları:** Veri setinin bulunduğu dizinleri ve CSV dosyasını dinamik olarak belirler.
2.  **Veri Dönüşümleri (Transforms):**
    * *Eğitim Seti:* Modelin ezberlemesini (overfitting) önlemek için **Data Augmentation** (Rastgele döndürme, renk değişimi vb.) uygulanır.
    * *Validasyon/Test Seti:* Sadece boyutlandırma ve normalizasyon yapılır; veri bozulmaz.
3.  **Veri Yükleme:**
    * Eğitim verisi `ImageFolder` ile yüklenir.
    * Eğitim verisi, `%80 Eğitim` ve `%20 Validasyon` olacak şekilde rastgele ikiye ayrılır (`random_split`).
    * Test verisi, yukarıda yazdığımız özel `GTSRBTestDataset` sınıfı ile yüklenir.
4.  **Batchleme:** Veriler karıştırılır (shuffle) ve modelin işleyebileceği paketler (batch) haline getirilir.

In [3]:
# --- ANA FONKSİYON: VERİ YÜKLEYİCİLERİ AL ---
def get_data_loaders(batch_size=64, num_workers=0):
    """
    Train, Validation ve Test veri yükleyicilerini ve sınıf sayısını döndürür.
    """
    # --- 1. Yollar ---
    try:
        BASE_DIR = Path(__file__).resolve().parent
    except NameError:
        BASE_DIR = Path('.').resolve() # İnteraktif pencere için yedek

    TRAIN_DIR = BASE_DIR / 'GTSRB' / 'Final_Training' / 'Images'
    TEST_DIR = BASE_DIR / 'GTSRB_Final_Test' / 'Final_Test' / 'Images'
    TEST_CSV_PATH = BASE_DIR / 'GT-final_test.csv'
    
    # Yolları kontrol et
    if not TRAIN_DIR.exists() or not TEST_DIR.exists() or not TEST_CSV_PATH.exists():
        print("--- HATA: Dosya Yolu Bulunamadı! ---")
        print(f"Aranan TRAIN_DIR: {TRAIN_DIR} -> Var mı? {TRAIN_DIR.exists()}")
        print(f"Aranan TEST_DIR: {TEST_DIR} -> Var mı? {TEST_DIR.exists()}")
        print(f"Aranan TEST_CSV: {TEST_CSV_PATH} -> Var mı? {TEST_CSV_PATH.exists()}")
        return None, None, None, -1 # Hata durumunda None döndür

    IMG_SIZE = 64
    VAL_SPLIT = 0.2
     # --- 2. Dönüşümler (Transforms) ---
    train_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomRotation(10), # Data augmentation
        transforms.ColorJitter(brightness=0.2, contrast=0.2), # Data augmentation
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    val_test_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)), # Augmentation YOK
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
 # --- 4. Veri Setlerini Yükleme ve Ayırma ---
    try:
        # A. Eğitim ve Validasyon
        full_train_dataset = torchvision.datasets.ImageFolder(
            root=str(TRAIN_DIR),
            transform=train_transforms
        )
        
        NUM_CLASSES = len(full_train_dataset.classes)
        print(f"Eğitim verisi yüklendi. Toplam {NUM_CLASSES} sınıf bulundu.")

        train_size = int((1.0 - VAL_SPLIT) * len(full_train_dataset))
        val_size = len(full_train_dataset) - train_size
        train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
        
        # Validasyon setinin transformunu augmentation'sız olanla değiştir
        val_dataset.dataset.transform = val_test_transforms
        print(f"Train/Validation ayrımı yapıldı: {len(train_dataset)} Train, {len(val_dataset)} Validation")

        # B. Test
        test_dataset = GTSRBTestDataset(
            csv_file=TEST_CSV_PATH,
            img_dir=TEST_DIR,
            transform=val_test_transforms
        )
        print(f"Test verisi yüklendi. Boyut: {len(test_dataset)}")

        # --- 5. DataLoaders Oluşturma ---
        train_loader = DataLoader(
            dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
        )
        val_loader = DataLoader(
            dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
        )
        test_loader = DataLoader(
            dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
        )
        
        print("DataLoaders (train, val, test) başarıyla oluşturuldu.")
        
        # Fonksiyon, bu 4 önemli değişkeni 'return' eder (geri döndürür)
        return train_loader, val_loader, test_loader, NUM_CLASSES

    except Exception as e:
        print(f"\nVeri yüklenirken beklenmedik bir hata oluştu: {e}")
        return None, None, None, -1

# 4. Fonksiyonun Test Edilmesi ve Kontrol

Bu blok, yazdığımız veri yükleme fonksiyonunun (`get_data_loaders`) hatasız çalışıp çalışmadığını kontrol eder.
* Veri yükleyicilerin başarıyla oluşturulup oluşturulmadığına bakar.
* Eğitim, validasyon ve test setlerindeki veri sayılarını (batch sayısı) ekrana basar.
* Örnek bir veri paketinin boyutlarını (Tensor shape) kontrol ederek boyut uyuşmazlığı olup olmadığını doğrular.

In [4]:
# --- Test Bloğu ---
# Bu dosya DOĞRUDAN çalıştırılırsa (import edilmek yerine),
# veri yüklemeyi test et.
if __name__ == "__main__":
    print("--- Veri_Duzenleme.ipynb Test Modunda Çalıştırılıyor ---")
    
    # 64 batch ve 0 worker ile fonksiyonu test et
    train_ldr, val_ldr, test_ldr, num_c = get_data_loaders(batch_size=64, num_workers=0)
    
    if train_ldr:
        print(f"\nTest başarılı. Sınıf sayısı: {num_c}")
        print(f"Train loader'da {len(train_ldr)} batch var.")
        print(f"Val loader'da {len(val_ldr)} batch var.")
        print(f"Test loader'da {len(test_ldr)} batch var.")
        
        # İlk batch'i almayı dene
        try:
            data, labels = next(iter(train_ldr))
            print(f"İlk train batch boyutu (Resimler): {data.shape}")
            print(f"İlk train batch boyutu (Etiketler): {labels.shape}")
        except Exception as e:
            print(f"Batch alınırken hata: {e}")
    else:
        print("\nTest başarısız. Veri yükleyiciler oluşturulamadı.")

--- Veri_Duzenleme.ipynb Test Modunda Çalıştırılıyor ---
Eğitim verisi yüklendi. Toplam 43 sınıf bulundu.
Train/Validation ayrımı yapıldı: 31367 Train, 7842 Validation
Test verisi yüklendi. Boyut: 12630
DataLoaders (train, val, test) başarıyla oluşturuldu.

Test başarılı. Sınıf sayısı: 43
Train loader'da 491 batch var.
Val loader'da 123 batch var.
Test loader'da 198 batch var.
İlk train batch boyutu (Resimler): torch.Size([64, 3, 64, 64])
İlk train batch boyutu (Etiketler): torch.Size([64])
